# MA_DL : Codierung 64bit

In [2]:
!pip install facenet-pytorch # due to issues with the google cloud service

In [3]:
import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models
from PIL import Image

from facenet_pytorch import MTCNN, InceptionResnetV1

dtype = torch.cuda.FloatTensor

## Model

In [4]:
class MultiLabel(nn.Sequential):
    def __init__(self, input_dim=512, output_dim=64):
        super(MultiLabel, self).__init__()
        self.l1 = nn.Linear(input_dim, input_dim)
        self.l2 = nn.Linear(input_dim, input_dim)
        self.l5 = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l5(x)
        return torch.sigmoid(x)

In [5]:
model = MultiLabel()
model.cuda()

MultiLabel(
  (l1): Linear(in_features=512, out_features=512, bias=True)
  (l2): Linear(in_features=512, out_features=512, bias=True)
  (l5): Linear(in_features=512, out_features=64, bias=True)
)

## Data

In [6]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()
to_image = transforms.ToPILImage()
transform = transforms.Compose([
    scaler, 
    to_tensor,
#    normalize
])

In [7]:
train_ds = datasets.ImageFolder('../../data/small_data/train/', transform=transform)
train_ds.idx_to_class = {i:c for c, i in train_ds.class_to_idx.items()}
train_dl = DataLoader(train_ds, batch_size=1, num_workers=4)

valid_ds = datasets.ImageFolder('../../data/small_data/valid/', transform=transform)
valid_ds.idx_to_class = {i:c for c, i in valid_ds.class_to_idx.items()}
valid_dl = DataLoader(valid_ds, batch_size=1, num_workers=4)

## Gesichtserkennung

In [9]:
mtcnn = MTCNN(
    image_size=160,
    thresholds=[0.6, 0.7, 0.7],
#        factor=0.709,
#        prewhiten=True,
    keep_all=True,
    device=torch.device('cuda')
    )

## Embeddings 

In [10]:
def get_vectors(imgs):
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(imgs).type(dtype)  
    
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(imgs.shape[0], 512)    

    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
      my_embedding.copy_(o.data.squeeze())    
    
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)    
    # 6. Run the model on our transformed image
    resnet(t_img)    
    # 7. Detach our copy function from the layer
    h.remove()    
    # 8. Return the feature vector
    return my_embedding

In [ ]:
# Alternativ resnet
vggf_resnet = InceptionResnetV1(pretrained='vggface2').eval()
img_embedding = vggf_resnet(img_cropped.unsqueeze(0))

In [11]:
# Load the pretrained model
resnet = models.resnet18(pretrained=True)
resnet.cuda()
# Use the model object to select the desired layer
layer = resnet._modules.get('avgpool')
_ = resnet.eval()

## Training

In [12]:
def calculate_mistakes(pred, target):
    #pred = pred[0]
#    print('calculating mistakes : ')
#    print('   pred: ' + str(pred.shape))
#    print('   target: ' + str(target.shape))
    
    mistakes = 0
    
    if len(pred) != len(target):
        raise Exception('sizes of both tensors must match')
        
    for x,y in zip(pred, target):
        if round(x.item()) != y.item():
            mistakes = mistakes + 1
        
    return mistakes

In [13]:
loss_func = nn.BCELoss() #MultiLabelSoftMarginLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=1e-5) #weight_decay=1e-4 

In [12]:
import matplotlib.pyplot as plt

In [15]:
for d, t in train_dl:
    print(d.shape)
    print(d)
    
    

torch.Size([1, 3, 224, 224])
tensor([[[[0.0118, 0.0118, 0.0118,  ..., 0.0078, 0.0118, 0.0118],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.1922, 0.2078, 0.2392,  ..., 0.2392, 0.2431, 0.2471],
          [0.2353, 0.2275, 0.2392,  ..., 0.2431, 0.2471, 0.2510],
          [0.2588, 0.2431, 0.2392,  ..., 0.2431, 0.2471, 0.2510]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0039, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0039, 0.0039,  ..., 0.0000, 0.0039, 0.0039],
          ...,
          [0.1765, 0.1804, 0.1882,  ..., 0.1843, 0.1882, 0.1922],
          [0.2078, 0.1843, 0.1843,  ..., 0.1882, 0.1922, 0.1961],
          [0.2157, 0.1843, 0.1804,  ..., 0.1882, 0.1922, 0.1961]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0039, 0.0039,  ..., 0.0078, 0.0078, 0.0078],
          [0.

torch.Size([1, 3, 224, 224])
tensor([[[[0.7373, 0.7294, 0.7333,  ..., 0.4745, 0.4863, 0.4980],
          [0.7490, 0.7373, 0.7412,  ..., 0.4902, 0.5098, 0.5294],
          [0.7608, 0.7451, 0.7490,  ..., 0.5098, 0.5373, 0.5569],
          ...,
          [0.2039, 0.1490, 0.1059,  ..., 0.5686, 0.5490, 0.5490],
          [0.2157, 0.1451, 0.1137,  ..., 0.5765, 0.5569, 0.5412],
          [0.2157, 0.1451, 0.1137,  ..., 0.5804, 0.5647, 0.5451]],

         [[0.5412, 0.5373, 0.5451,  ..., 0.4706, 0.4824, 0.4902],
          [0.5529, 0.5451, 0.5569,  ..., 0.4863, 0.5059, 0.5216],
          [0.5647, 0.5529, 0.5647,  ..., 0.5059, 0.5333, 0.5529],
          ...,
          [0.2039, 0.1490, 0.1059,  ..., 0.6235, 0.5961, 0.5843],
          [0.2157, 0.1451, 0.1098,  ..., 0.6314, 0.6039, 0.5725],
          [0.2118, 0.1412, 0.1098,  ..., 0.6353, 0.6039, 0.5725]],

         [[0.2510, 0.2353, 0.2353,  ..., 0.2980, 0.3176, 0.3294],
          [0.2627, 0.2431, 0.2431,  ..., 0.3137, 0.3373, 0.3569],
          [0.

tensor([[[[0.3451, 0.3412, 0.3412,  ..., 0.7725, 0.7765, 0.7765],
          [0.3216, 0.3176, 0.3176,  ..., 0.7569, 0.7608, 0.7608],
          [0.2863, 0.2824, 0.2863,  ..., 0.7373, 0.7412, 0.7412],
          ...,
          [0.0196, 0.0235, 0.0196,  ..., 0.0588, 0.0510, 0.0510],
          [0.0235, 0.0196, 0.0118,  ..., 0.0627, 0.0549, 0.0549],
          [0.0275, 0.0235, 0.0118,  ..., 0.0627, 0.0549, 0.0549]],

         [[0.2392, 0.2353, 0.2314,  ..., 0.7490, 0.7529, 0.7569],
          [0.2157, 0.2118, 0.2078,  ..., 0.7412, 0.7490, 0.7490],
          [0.1882, 0.1804, 0.1804,  ..., 0.7294, 0.7373, 0.7373],
          ...,
          [0.0196, 0.0235, 0.0314,  ..., 0.0588, 0.0510, 0.0510],
          [0.0235, 0.0235, 0.0235,  ..., 0.0627, 0.0549, 0.0588],
          [0.0275, 0.0275, 0.0235,  ..., 0.0627, 0.0588, 0.0588]],

         [[0.1569, 0.1529, 0.1490,  ..., 0.6078, 0.6118, 0.6078],
          [0.1373, 0.1333, 0.1294,  ..., 0.5961, 0.6000, 0.6000],
          [0.1098, 0.1059, 0.1059,  ..., 0

tensor([[[[0.3490, 0.3490, 0.3490,  ..., 0.3216, 0.3216, 0.3216],
          [0.3647, 0.3608, 0.3569,  ..., 0.3255, 0.3255, 0.3255],
          [0.3804, 0.3765, 0.3686,  ..., 0.3333, 0.3294, 0.3294],
          ...,
          [0.2000, 0.2000, 0.1961,  ..., 0.1647, 0.1647, 0.1647],
          [0.1961, 0.1961, 0.1922,  ..., 0.1569, 0.1529, 0.1490],
          [0.1961, 0.1961, 0.1922,  ..., 0.1569, 0.1451, 0.1412]],

         [[0.2627, 0.2627, 0.2627,  ..., 0.2549, 0.2549, 0.2549],
          [0.2784, 0.2745, 0.2706,  ..., 0.2588, 0.2588, 0.2588],
          [0.2941, 0.2902, 0.2824,  ..., 0.2667, 0.2627, 0.2627],
          ...,
          [0.1765, 0.1765, 0.1725,  ..., 0.1451, 0.1451, 0.1451],
          [0.1725, 0.1725, 0.1686,  ..., 0.1373, 0.1333, 0.1294],
          [0.1725, 0.1725, 0.1686,  ..., 0.1373, 0.1255, 0.1216]],

         [[0.1725, 0.1725, 0.1725,  ..., 0.2157, 0.2157, 0.2157],
          [0.1882, 0.1843, 0.1843,  ..., 0.2196, 0.2196, 0.2196],
          [0.2118, 0.2078, 0.2000,  ..., 0

torch.Size([1, 3, 224, 224])
tensor([[[[0.0627, 0.0627, 0.0627,  ..., 0.1255, 0.1255, 0.1216],
          [0.0627, 0.0627, 0.0627,  ..., 0.1451, 0.1294, 0.1216],
          [0.0627, 0.0627, 0.0627,  ..., 0.1529, 0.1373, 0.1294],
          ...,
          [0.1373, 0.1412, 0.1373,  ..., 0.1686, 0.1882, 0.2000],
          [0.1412, 0.1412, 0.1373,  ..., 0.1647, 0.1843, 0.2039],
          [0.1373, 0.1373, 0.1373,  ..., 0.1647, 0.1843, 0.2000]],

         [[0.0784, 0.0784, 0.0784,  ..., 0.0784, 0.0784, 0.0745],
          [0.0784, 0.0784, 0.0784,  ..., 0.1059, 0.0863, 0.0784],
          [0.0784, 0.0784, 0.0784,  ..., 0.1176, 0.0980, 0.0941],
          ...,
          [0.1098, 0.1137, 0.1137,  ..., 0.1608, 0.1804, 0.1882],
          [0.1098, 0.1098, 0.1137,  ..., 0.1608, 0.1804, 0.1961],
          [0.1137, 0.1137, 0.1137,  ..., 0.1608, 0.1804, 0.1961]],

         [[0.0824, 0.0824, 0.0824,  ..., 0.0784, 0.0784, 0.0745],
          [0.0824, 0.0824, 0.0824,  ..., 0.1020, 0.0824, 0.0745],
          [0.

torch.Size([1, 3, 224, 224])
tensor([[[[0.1137, 0.1216, 0.1294,  ..., 0.2392, 0.1804, 0.1490],
          [0.0941, 0.0980, 0.1020,  ..., 0.2118, 0.1569, 0.1255],
          [0.0706, 0.0784, 0.0863,  ..., 0.1804, 0.1216, 0.0863],
          ...,
          [0.1255, 0.1255, 0.1255,  ..., 0.1216, 0.1255, 0.1255],
          [0.1373, 0.1333, 0.1333,  ..., 0.1294, 0.1333, 0.1333],
          [0.1412, 0.1373, 0.1333,  ..., 0.1294, 0.1451, 0.1490]],

         [[0.1216, 0.1333, 0.1451,  ..., 0.2510, 0.1961, 0.1725],
          [0.1176, 0.1176, 0.1255,  ..., 0.2353, 0.1882, 0.1647],
          [0.1098, 0.1176, 0.1255,  ..., 0.2196, 0.1725, 0.1412],
          ...,
          [0.1529, 0.1529, 0.1529,  ..., 0.2039, 0.2078, 0.2039],
          [0.1647, 0.1608, 0.1608,  ..., 0.2039, 0.2118, 0.2078],
          [0.1686, 0.1647, 0.1608,  ..., 0.2000, 0.2196, 0.2196]],

         [[0.0667, 0.0667, 0.0745,  ..., 0.1686, 0.1216, 0.0941],
          [0.0549, 0.0471, 0.0510,  ..., 0.1529, 0.1098, 0.0824],
          [0.

torch.Size([1, 3, 224, 224])
tensor([[[[0.0275, 0.0353, 0.0314,  ..., 0.8510, 0.8471, 0.8353],
          [0.0275, 0.0314, 0.0314,  ..., 0.8392, 0.8314, 0.8196],
          [0.0235, 0.0314, 0.0314,  ..., 0.8196, 0.8078, 0.8000],
          ...,
          [0.9294, 0.9294, 0.9294,  ..., 0.9765, 0.9765, 0.9765],
          [0.9294, 0.9294, 0.9294,  ..., 0.9765, 0.9765, 0.9765],
          [0.9294, 0.9294, 0.9294,  ..., 0.9765, 0.9765, 0.9765]],

         [[0.0392, 0.0471, 0.0510,  ..., 0.8824, 0.8706, 0.8549],
          [0.0392, 0.0510, 0.0510,  ..., 0.8706, 0.8549, 0.8392],
          [0.0392, 0.0471, 0.0510,  ..., 0.8588, 0.8392, 0.8235],
          ...,
          [0.9922, 0.9922, 0.9922,  ..., 1.0000, 1.0000, 1.0000],
          [0.9922, 0.9922, 0.9922,  ..., 1.0000, 1.0000, 1.0000],
          [0.9922, 0.9922, 0.9922,  ..., 1.0000, 1.0000, 1.0000]],

         [[0.0588, 0.0667, 0.0667,  ..., 0.8039, 0.7843, 0.7686],
          [0.0667, 0.0745, 0.0706,  ..., 0.8039, 0.7804, 0.7608],
          [0.

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0314, 0.0314, 0.0314,  ..., 0.0353, 0.0314, 0.0235],
          [0.0314, 0.0275, 0.0275,  ..., 0.0235, 0.0235, 0.0196],
          [0.0314, 0.0275, 0.0275,  ..., 0.0196, 0.0196, 0.0196]],

         [[0.0039, 0.0039, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0314, 0.0314, 0.0314,  ..., 0.0314, 0.0314, 0.0235],
          [0.0314, 0.0275, 0.0275,  ..., 0.0196, 0.0235, 0.0196],
          [0.0314, 0.0275, 0.0275,  ..., 0.0157, 0.0196, 0.0196]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0078, 0.0078, 0.0078],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

tensor([[[[0.8902, 0.8902, 0.8980,  ..., 0.9882, 0.9843, 0.9843],
          [0.9059, 0.9059, 0.9137,  ..., 0.9882, 0.9882, 0.9882],
          [0.9333, 0.9333, 0.9373,  ..., 0.9961, 0.9961, 0.9961],
          ...,
          [0.6314, 0.6706, 0.6824,  ..., 0.0196, 0.0196, 0.0196],
          [0.6157, 0.6549, 0.6627,  ..., 0.0196, 0.0235, 0.0235],
          [0.5961, 0.6314, 0.6353,  ..., 0.0196, 0.0235, 0.0235]],

         [[0.8314, 0.8314, 0.8353,  ..., 1.0000, 0.9961, 0.9961],
          [0.8471, 0.8471, 0.8510,  ..., 0.9961, 0.9961, 0.9961],
          [0.8745, 0.8745, 0.8745,  ..., 1.0000, 0.9961, 0.9961],
          ...,
          [0.4588, 0.5020, 0.5255,  ..., 0.0627, 0.0627, 0.0627],
          [0.4314, 0.4784, 0.5020,  ..., 0.0627, 0.0667, 0.0667],
          [0.4039, 0.4510, 0.4706,  ..., 0.0627, 0.0667, 0.0667]],

         [[0.7176, 0.7176, 0.7294,  ..., 0.9608, 0.9569, 0.9569],
          [0.7333, 0.7333, 0.7412,  ..., 0.9451, 0.9451, 0.9451],
          [0.7608, 0.7608, 0.7647,  ..., 0

In [16]:
epochs = 1000
for e in range(epochs):
    print("======================")
    print("Epoch : " + str(e))
    epoch_loss = 0
    epoch_mistakes = 0
    train_size = len(train_dl)
    
    # activate train mode
    model.train()

    for index, (data, target) in enumerate(train_dl):
        
        faces, prob = mtcnn(to_image(data[0]), return_prob=True)
 
 #       if faces is None:
 #           print('no face')
 #           continue
 #       else:
 #           print('face found')
                
        target_t = torch.cuda.FloatTensor([int(x) for x in train_ds.idx_to_class[target[0].item()]])

        emb = get_vectors(faces)    
        data_v   = Variable(emb[0], requires_grad=False).type(dtype)
        target_v = Variable(target_t, requires_grad=False).type(dtype)
        print(data_v)
        
        # forward
#        print('datav: ' + str(data_v.shape))
        pred = model.forward(data_v)
#        print('pred: ' + str(pred.shape))
        # zero grads
        optimizer.zero_grad()
        # calculate loss
        loss = loss_func(pred, target_v.float())
        epoch_loss = epoch_loss + loss
        # back prop
        loss.backward()
        optimizer.step()
        
        epoch_mistakes = epoch_mistakes + calculate_mistakes(pred, target_t)
        
    print("Train loss : " + str(epoch_loss.item()))
    print("Average errors : " + str(epoch_mistakes/train_size))
    
    # activate eval mode
    model.eval()
    
    valid_loss = 0
    valid_mistakes = 0
    valid_size = len(valid_dl)
    for index, (data, target) in enumerate(valid_dl):
        faces, prob = mtcnn(to_image(data[0]), return_prob=True)
        target_t = torch.FloatTensor([int(x) for x in valid_ds.idx_to_class[target[0].item()]])
        
        emb = get_vectors(faces) 
        data_v   = Variable(emb[0], requires_grad=False).type(dtype)
        target_v = Variable(target_t, requires_grad=False).type(dtype)

        pred = model.forward(data_v)
        loss = loss_func(pred, target_v.float())
        valid_loss = valid_loss + loss
        valid_mistakes = valid_mistakes + calculate_mistakes(pred, target_t)
        
        
    print("Valid loss : " + str(valid_loss.item()))
    print("Average valid errors : " + str(valid_mistakes/valid_size))

Epoch : 0
tensor([2.5640e-01, 1.3146e+00, 1.5079e+00, 3.1035e-01, 2.0382e-03, 1.3857e-01,
        5.9693e-01, 6.1075e-01, 0.0000e+00, 0.0000e+00, 1.1364e+00, 1.3891e+00,
        1.5071e-02, 1.4342e-02, 4.7746e-01, 1.7059e+00, 1.0079e+00, 4.1452e-01,
        8.5286e-03, 9.8521e-01, 1.8991e-03, 1.2539e-01, 0.0000e+00, 4.7227e-01,
        3.2950e-01, 0.0000e+00, 4.9375e-02, 2.1689e-01, 1.9052e-01, 1.9319e-01,
        1.4432e-01, 2.7139e-01, 3.9479e-01, 1.2855e+00, 2.2850e-01, 1.9375e-01,
        1.1812e+00, 2.1384e-01, 4.9124e+00, 0.0000e+00, 1.6862e+00, 2.8418e+00,
        6.1418e-01, 8.8550e-04, 2.1597e+00, 1.9472e+00, 2.1129e-01, 2.2915e+00,
        1.8560e+00, 1.3147e+00, 5.7089e-01, 2.8221e+00, 8.1840e-02, 2.1415e+00,
        1.2533e-01, 1.1812e-01, 7.0330e-02, 5.4673e-01, 1.3466e+00, 2.4492e+00,
        2.7128e-02, 7.1513e-01, 2.0384e+00, 1.1267e+00, 2.1586e-01, 9.7823e-01,
        1.4201e+00, 1.3790e-01, 1.6494e-01, 8.1740e-01, 6.1346e-01, 2.9539e-01,
        8.1662e-02, 3.6244e+00

tensor([2.0979e-01, 1.4664e+00, 1.1818e+00, 5.6739e-02, 8.1431e-03, 1.5621e-01,
        4.0723e-01, 9.6782e-01, 2.7138e-02, 5.7206e-03, 7.5279e-01, 2.4701e-01,
        1.1890e-02, 1.0037e-01, 4.5112e-01, 1.4492e+00, 5.3997e-01, 5.0954e-01,
        4.3234e-02, 1.7924e+00, 1.5589e-01, 2.6706e-01, 5.3948e-02, 5.6143e-02,
        6.8188e-01, 1.1802e-02, 6.0260e-01, 2.0227e-01, 1.3838e-01, 8.3826e-02,
        3.6079e-01, 5.3247e-02, 6.6667e-01, 1.5361e+00, 3.0354e-02, 8.9931e-02,
        2.9347e-01, 3.0022e-01, 2.6416e+00, 0.0000e+00, 9.2220e-01, 1.3575e+00,
        5.8849e-01, 1.6351e-01, 1.9557e+00, 1.9061e+00, 5.3215e-02, 2.4424e+00,
        1.4181e+00, 2.5264e+00, 1.1711e-02, 1.9415e+00, 2.1326e-01, 2.0856e+00,
        3.9832e-02, 6.3539e-01, 9.9250e-04, 8.4061e-01, 4.2248e-01, 1.9659e+00,
        3.2558e-01, 7.7374e-01, 2.3061e+00, 2.8738e-01, 4.8297e-01, 1.4353e+00,
        1.2465e+00, 2.1924e-01, 1.6645e-01, 5.0196e-01, 6.3223e-01, 1.7139e-01,
        1.7240e-01, 2.5903e+00, 1.6799e+

tensor([1.4358e+00, 2.2440e-01, 9.4299e-01, 3.3737e-01, 0.0000e+00, 8.0527e-02,
        3.7701e-01, 1.6512e-01, 8.4105e-02, 4.6577e-01, 1.8275e-02, 1.6169e-01,
        2.2847e-01, 0.0000e+00, 1.3323e-01, 3.4556e-01, 5.3522e-01, 4.2191e-01,
        9.1297e-02, 8.2701e-01, 4.8211e-01, 6.3058e-01, 0.0000e+00, 0.0000e+00,
        2.8210e-01, 3.6422e-02, 4.7395e-02, 1.0991e-01, 4.2538e-01, 1.4894e-01,
        6.3885e-01, 2.9303e-01, 5.7908e-03, 3.8097e+00, 1.0312e-01, 9.2663e-02,
        1.6726e+00, 1.9208e-01, 2.9104e+00, 0.0000e+00, 5.7659e-01, 8.7978e-01,
        4.0430e-01, 1.6411e-01, 8.0618e-01, 9.6064e-01, 3.7377e-01, 1.4195e+00,
        1.5427e+00, 4.6605e-01, 5.6683e-01, 1.4538e+00, 4.3050e-01, 1.0589e+00,
        1.0866e-01, 2.5572e-02, 0.0000e+00, 1.5086e+00, 3.6656e-01, 1.9255e+00,
        2.4152e-01, 7.2502e-01, 2.0521e+00, 5.8283e-02, 2.3426e-01, 1.3483e+00,
        4.0072e-01, 0.0000e+00, 1.4372e-01, 1.6026e-01, 1.2244e+00, 0.0000e+00,
        3.4356e-01, 2.2518e+00, 1.0029e+

tensor([1.2098e-01, 5.6710e-01, 4.5912e-01, 6.6963e-02, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 3.2198e-01, 0.0000e+00, 2.2327e-03, 2.3194e-01, 1.5353e-01,
        1.5692e-01, 1.3000e-02, 2.4520e-03, 5.4555e-01, 1.5230e+00, 1.9669e-01,
        3.5374e-02, 9.1406e-02, 2.3580e-01, 4.1996e-01, 5.9927e-04, 1.2275e-01,
        5.2359e-01, 2.7097e-01, 6.6697e-01, 6.6234e-02, 6.6653e-01, 7.6735e-02,
        8.2563e-02, 4.4280e-01, 6.3934e-02, 9.9109e-01, 3.7779e-01, 4.2376e-01,
        1.3463e+00, 0.0000e+00, 2.5735e+00, 0.0000e+00, 5.9693e-01, 2.0685e-01,
        1.5802e-01, 6.1010e-02, 1.4022e+00, 6.1624e-01, 4.1726e-02, 2.1329e+00,
        1.0752e+00, 1.4025e+00, 4.9172e-01, 7.4965e-01, 1.2792e-01, 7.6816e-01,
        2.4649e-02, 2.3595e-01, 7.3164e-02, 1.6169e+00, 9.8771e-01, 1.3200e+00,
        6.9784e-01, 7.0467e-02, 7.5217e-01, 5.0295e-01, 5.7090e-02, 2.9061e-01,
        2.5660e-01, 1.8813e-01, 1.9374e-02, 6.5763e-01, 6.0851e-02, 3.1505e-02,
        1.5824e-01, 1.4278e+00, 1.2947e+

tensor([3.1510e-01, 7.8257e-01, 5.4543e-01, 1.8619e-01, 3.4061e-02, 7.3207e-03,
        8.3518e-01, 4.5125e-01, 1.8608e-01, 1.7746e-01, 1.3976e-01, 2.0070e-01,
        2.5121e-01, 2.8557e-02, 4.4484e-01, 1.0760e+00, 1.4938e+00, 6.1821e-01,
        3.0019e-01, 3.8607e-01, 1.5844e-02, 1.0409e-01, 2.0622e-03, 3.3411e-02,
        1.2251e+00, 7.7172e-02, 5.3802e-01, 5.8176e-02, 1.5590e+00, 9.3380e-02,
        1.2102e+00, 2.8653e-01, 1.1910e-01, 9.3985e-01, 5.8515e-01, 2.1172e-01,
        1.1869e+00, 3.0320e-01, 2.6213e+00, 3.8941e-02, 5.7753e-01, 1.1637e-01,
        1.2807e+00, 2.9093e-01, 1.5871e+00, 2.1664e+00, 4.5639e-01, 2.6448e+00,
        1.9928e+00, 6.2512e-01, 5.5283e-01, 5.5620e-01, 4.2160e-01, 1.5570e+00,
        4.0581e-02, 1.8464e-02, 0.0000e+00, 2.5402e+00, 1.9721e-02, 1.4063e+00,
        9.9674e-02, 1.8394e+00, 8.7321e-01, 1.9286e-02, 0.0000e+00, 4.5749e-01,
        1.4425e+00, 4.1821e-02, 6.7278e-02, 3.7145e-01, 1.2846e-01, 6.9442e-01,
        9.4971e-01, 1.0360e+00, 6.7607e-

...

tensor([8.4131e-01, 4.1303e-01, 7.4612e-01, 0.0000e+00, 2.0016e-01, 0.0000e+00,
        8.0653e-02, 1.2134e-02, 0.0000e+00, 5.1919e-02, 1.9107e-01, 1.0169e-02,
        4.1748e-04, 0.0000e+00, 1.8093e-01, 4.1177e-01, 8.3978e-01, 4.8409e-02,
        9.6720e-02, 1.3895e+00, 0.0000e+00, 5.8173e-01, 4.7907e-02, 6.8268e-03,
        6.7802e-01, 0.0000e+00, 6.8680e-01, 0.0000e+00, 7.2305e-01, 6.4694e-04,
        1.6690e-01, 1.5381e+00, 1.1485e+00, 2.0027e+00, 5.2097e-01, 1.6063e-01,
        8.1987e-01, 0.0000e+00, 1.3553e+00, 7.6623e-02, 6.3145e-01, 6.9275e-01,
        4.1493e-01, 1.2342e-02, 1.8765e+00, 1.5044e+00, 3.2220e-02, 6.4344e-01,
        1.4261e+00, 4.0826e+00, 1.9896e-02, 9.5946e-01, 5.7080e-01, 1.0094e+00,
        0.0000e+00, 1.1248e-01, 2.6487e-01, 2.0012e+00, 4.6546e-01, 1.6891e+00,
        1.4844e-01, 3.0408e-01, 1.4096e+00, 6.4545e-02, 0.0000e+00, 6.4287e-01,
        7.6621e-01, 0.0000e+00, 8.1730e-03, 2.8699e-01, 4.9066e-01, 8.6223e-03,
        2.0056e-01, 3.6321e+00, 1.5128e+

tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 7.5284e-02, 0.0000e+00, 4.9152e-01,
        4.7061e-01, 4.3663e+00, 3.8290e-01, 2.8501e-03, 1.4280e-03, 8.6560e-02,
        0.0000e+00, 6.3701e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.5639e+00,
        1.6239e-01, 2.3242e-02, 3.9855e-02, 1.2880e+00, 2.4051e-03, 0.0000e+00,
        8.1616e-02, 2.3130e-02, 2.0957e-01, 0.0000e+00, 2.6232e-01, 5.4061e-01,
        2.0224e-03, 2.9990e-01, 6.8183e-02, 4.9672e-01, 1.5829e-01, 2.1604e-01,
        7.6264e-01, 9.3135e-02, 2.3422e-04, 3.0326e-01, 3.3828e-03, 0.0000e+00,
        9.9503e-01, 2.0293e+00, 5.7889e-01, 8.2593e-04, 1.2383e-01, 1.4249e-01,
        3.0226e-01, 3.2815e-01, 7.4375e-01, 0.0000e+00, 6.7731e-02, 6.1603e-04,
        0.0000e+00, 1.2758e-02, 4.7874e-01, 1.7830e+00, 7.7750e-01, 2.0010e-01,
        1.0839e+00, 1.2117e-03, 1.0925e-01, 0.0000e+00, 4.8858e-02, 1.7961e-01,
        3.0016e-02, 2.1937e-01, 1.3483e-01, 7.0640e-01, 8.4350e-02, 2.2053e-01,
        0.0000e+00, 4.1946e-01, 1.0059e+

tensor([2.3214e-01, 1.0479e+00, 9.1295e-01, 1.5611e-03, 3.6018e-01, 5.9331e-01,
        2.1669e-01, 1.2584e+00, 2.2979e-01, 1.3495e-02, 6.6292e-01, 5.0996e-01,
        2.5833e-01, 4.7968e-01, 1.9524e-01, 1.8217e+00, 8.8965e-01, 2.3385e-01,
        2.8878e-02, 6.5798e-01, 4.3542e-02, 7.2137e-01, 1.9989e-01, 6.7514e-01,
        1.0588e+00, 9.5926e-03, 6.9828e-01, 1.2773e-01, 5.0848e-01, 5.0207e-02,
        5.5310e-01, 4.9929e-01, 4.4793e-02, 9.7499e-01, 2.7202e-01, 5.3942e-02,
        1.4834e+00, 4.4694e-01, 4.1024e+00, 1.1918e-02, 1.3330e+00, 7.3084e-01,
        8.9745e-01, 1.9957e-01, 2.1903e+00, 2.6455e+00, 1.7506e-02, 1.1462e+00,
        1.4822e+00, 1.3905e+00, 6.7008e-01, 1.7429e+00, 1.1463e+00, 1.1570e+00,
        4.1713e-02, 6.0075e-02, 1.6918e-01, 4.0675e-01, 5.2242e-01, 2.1129e+00,
        1.2009e-01, 5.6790e-01, 2.7627e+00, 8.0788e-01, 3.4170e-01, 3.5695e-01,
        1.7049e+00, 4.4667e-01, 0.0000e+00, 3.0603e-01, 1.2712e+00, 3.4035e-01,
        5.1943e-03, 1.4113e+00, 1.3036e+

tensor([5.2002e-01, 8.0225e-01, 8.9415e-01, 7.0228e-02, 0.0000e+00, 1.2035e-02,
        2.3140e-01, 4.6775e-01, 1.7926e-02, 1.1690e-01, 5.8784e-01, 5.3466e-01,
        1.4676e-01, 3.2832e-02, 3.4453e-02, 2.0324e+00, 1.5827e+00, 4.4898e-03,
        3.8272e-02, 8.2132e-01, 0.0000e+00, 4.5226e-01, 1.4913e-02, 7.6779e-01,
        9.8372e-01, 0.0000e+00, 2.7568e-01, 4.2900e-01, 8.2054e-01, 2.1619e-01,
        3.3803e-01, 4.4896e-01, 2.2988e-01, 1.2142e+00, 5.8627e-01, 1.4274e-01,
        8.6238e-01, 4.6956e-01, 4.5679e+00, 1.4153e-03, 8.5723e-01, 1.2522e+00,
        1.1349e+00, 2.9917e-01, 2.9211e+00, 1.7813e+00, 1.5451e-01, 1.6283e+00,
        1.2830e+00, 1.1768e+00, 9.2325e-02, 2.1626e+00, 9.5154e-01, 2.6875e+00,
        1.5636e-02, 1.0661e+00, 1.3404e-01, 8.9553e-01, 2.1170e-02, 2.2640e+00,
        3.1583e-01, 4.2469e-01, 2.5622e+00, 4.6687e-01, 8.2670e-01, 6.1624e-01,
        3.4629e-01, 4.4820e-03, 7.0369e-02, 5.9154e-01, 1.6370e-01, 7.2521e-02,
        2.3625e-01, 2.4873e+00, 1.4773e+

KeyboardInterrupt: 

In [14]:
# torch.save(model.state_dict(), 'dl64b_v2_1000.pt')